# M5 Timeseries Clusterization with tsfresh
## Using DBSCAN, Mean shift, k-mean

This notebook is based on Alexander Alekseitsev [presentation](https://www.youtube.com/watch?v=WwNRRvYLGXI&t=1634s)
and uses [this](https://www.kaggle.com/sibmike/m5-out-of-stock-feature-640x-faster) notebook to find M5 out-of-stocks. It also has two goals: 
* show an **easy way to generate time series features** with [tsfresh](https://tsfresh.readthedocs.io/en/latest/index.html)
* use the generated features to **clusterize time-series**

Th plan is as follows:
### 0. Load datasets

### 1. Preprocessing:
* Cap 3 sigma outliers and then fill in stockouts with the mean sales in preceding 28 days.
* Log scale timeseries to further lower the impact of outliers
* Standardscaler to bring all time series to one scale
* Aggregate to weekly-product_id

### 2. Feature engineering (tsfresh):
* Trends
* Autocorrelcations
* Skewness
* Fourier transforms
* Wavelet transforms
* Holiday specific features

### 3. DBSCAN + Mean shift to find the number of clusters -> k-mean, then stack predictions.
* Some visualizations to see how the clusters differ & if they make sense.

We will not try to use the new feature, but will export it so you can try it out on your own.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from scipy.sparse import csr_matrix

from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import KNNImputer
from scipy import stats
from tqdm.notebook import tqdm

import gc
%matplotlib inline

In [2]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# M5 - Step 1: Clean

## Clean: Impute Gaps & Cap Outliers
#### Capping and imputing sales timeseries while preserving the target:

Sales are noised with outliers and long zero gaps. We split target into three timeseries: 

* **sale_outlier_diff** - differnece between outlier and maximum non outlier value; outliers are flaged using z score.
* **sale_cap_ipmuted** - Sales with capped outliers and imputed gap_days with last observed value.
* **imputed_gaps_1** & **imputed_gaps_23** - Store/Satte specific gaps imputed with flat mean

# 0. Load Datasets
All three datasets needed because we need to calculate sales in USD.

In [3]:
# DEFAULT datasets
data_pass = '/kaggle/input/m5-forecasting-accuracy/'

# Sales quantities:

# Calendar to get week number to join sell prices:
calendar = pd.read_csv(data_pass+'calendar.csv')
calendar.d = calendar.d.str.extract('(\d+)').astype(np.int16)
calendar = reduce_mem_usage(calendar)

# Sell prices to calculate sales in USD:
sell_prices = pd.read_csv(data_pass+'sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)

Mem. usage decreased to  0.11 Mb (44.3% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


### var explanation:
For complete explanation of how we got the variables take a look at ["M5 - 'out_of_stock' feature 640x faster"](https://www.kaggle.com/sibmike/m5-out-of-stock-feature-640x-faster). Here is brief explanation of variables:
* **release** - first time when product has been sold
* **gaps** - int, zero or not flag
* **gap_days** - int, number of days in sequence of consecutive zeroes
* **gap_e** - how many years needed to observe a gap of length 'gap_days'
* **sale_prob**[](http://) - probability of no sales

In [4]:
# GAPS datasets
gap_columns = ['id', 'item_id','store_id', 'dept_id', 'state_id', 'd', 'sales', 'release', 'gap_e', 'sale_prob']

# Load grid with identified stockouts:
file_pass = '../input/m5-out-of-stock-feature-640x-faster/grid_part_1_gaps.pkl'
grid_df = reduce_mem_usage(pd.read_pickle(file_pass)[gap_columns])

# Add week of year:
grid_df = pd.merge(
    grid_df, 
    calendar[['wm_yr_wk','event_name_1','d']], 
    how = 'left', 
    left_on = ['d'], 
    right_on = ['d']
)

# Add price:
grid_df = grid_df.merge(
    sell_prices, 
    on = ['store_id', 'item_id', 'wm_yr_wk'], 
    how = 'left'
)

grid_df.head()
del sell_prices, calendar

Mem. usage decreased to 1029.91 Mb (34.3% reduction)


In [5]:
# To select timeseries features useful for clusterization we will need a set of 'targets' to split them in two classes.
# We will use the 'store_id','dept_id','state_id' to do the splits:
targets = list(grid_df.dept_id.unique())#+ list(grid_df.state_id.unique())+list(grid_df.store_id.unique())
grid_df.drop(columns=['store_id','dept_id','state_id'])

,id,item_id,d,sales,release,gap_e,sale_prob,wm_yr_wk,event_name_1,sell_price
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,1,12.0,0,0.000000e+00,0.133911,11101,NaN,0.459961
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,1,2.0,0,5.738430e-03,0.477539,11101,NaN,1.559570
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,1,0.0,0,1.798078e-02,0.473877,11101,NaN,3.169922
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,1,0.0,0,4.059916e-03,0.674805,11101,NaN,5.980469
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,1,4.0,0,1.298860e-01,0.156128,11101,NaN,0.700195
...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,FOODS_3_823,1941,NaN,0,2.645121e+06,0.489258,11617,NaN,2.980469
46881673,FOODS_3_824_WI_3_validation,FOODS_3_824,1941,NaN,0,1.210731e+04,0.609375,11617,NaN,2.480469
46881674,FOODS_3_825_WI_3_validation,FOODS_3_825,1941,NaN,0,1.354414e+11,0.342041,11617,NaN,3.980469
46881675,FOODS_3_826_WI_3_validation,FOODS_3_826,1941,NaN,230,8.217696e+20,0.145874,11617,NaN,1.280273


In [6]:
# I use 100 years as threshold for a stockout. If you feel it is too much, lower it.
years_threshold = 100
grid_df.loc[:,'stockout'] = (grid_df.gap_e >= years_threshold)*1

# 1. Preprocessing:
### Cap 3 sigma outliers:

In [7]:
# Find outliers based on z score:
## Filter out values more than 0 as otherwise all sales might be flagged as outliers:
mask = grid_df.sales > 0

## Group by 'id' as sales values have diffrenet mean from store to store: 
df_grouped = grid_df[mask].groupby(['id'])

## Find indices of all outliers:
outlier_idx = []
for group_name, g in tqdm(df_grouped):
    outlier_idx += list(g[stats.zscore(g.sales) > 3].index) 
    
## Create separate column with outliers:
grid_df.loc[:,'sale_outlier'] = 0
grid_df.loc[outlier_idx,'sale_outlier']=grid_df.loc[outlier_idx,'sales']

/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:183: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)
/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [8]:
# Cap sales outliers with maximum non outlier value:
grid_df['sales_clean'] = grid_df['sales']
nan_idx = grid_df[~grid_df.sales.isna()].index
grid_df.loc[outlier_idx,'sales_clean'] = np.nan

grid_df.loc[nan_idx,'sales_clean'] = grid_df.loc[nan_idx,'sales_clean'].fillna(grid_df.groupby('id')['sales_clean'].transform('max'))

### ... and then fill in stockouts with last seen sales.

In [9]:
# Group by id:
df_grouped = grid_df.groupby(["id"])["sales_clean"]

# Generate rolling mean to fill in stockouts:
lag = 1
w = 28
col_name = f"rolling_w{str(w)}_l{str(lag)}_mean"
grid_df[col_name] = df_grouped.transform(lambda x: x.shift(lag).rolling(w).mean()).astype(np.float32)

# Fill in stockouts: 
mask = grid_df.stockout>0
grid_df.loc[mask, 'sales_clean'] = grid_df.loc[mask, col_name]

### Holiday features:

1. Calculate difference between rolling mean before and after the date.
2. Group by item_id, holiday
3. Use the differences adound different  holidays as features for item_ids.

This will help to **cluster products by holiday** like flowers are valentines day products and turkey is thanksgiving product.

In [10]:
holiday_list = ["Christmas",
"Easter",
"Halloween",
"IndependenceDay",
"LaborDay",
"MemorialDay",
"Mother's day",
"NewYear",
"SuperBowl",
"Thanksgiving",
"ValentinesDay"]

In [11]:
holiday_mask = grid_df.event_name_1.isin(holiday_list)

In [12]:
# Creating Holiday lift feature:
col_name = f"holiday_lift"
grid_df[col_name] = df_grouped.transform(lambda x: 
                                         (x.shift(8).rolling(7).mean() \
                                          - x.shift(-1).rolling(7).mean())\
                                         >=0)*(holiday_mask*1)

del df_grouped

### Log scale timeseries to further lower the impact of outliers


In [13]:
# Note we add 1 so that zeroes map to zeroes and small values map to small values.
grid_df.loc[:, 'sales_clean'] = np.log(grid_df.sales_clean+1)

### Standardscaler to bring all time series to one scale

In [14]:
# Please, note that this is implementation is one-way there is no way to go back to unscaled version.
grid_df['sales_clean'] = grid_df.groupby('id').sales_clean.transform(lambda x: scale(x.astype(float)))

# Week_df

In [15]:
# Create week-porduct_id aggregations:
week_df = grid_df[['item_id','wm_yr_wk','sales_clean']].groupby(['item_id','wm_yr_wk']).mean().fillna(0).astype(np.float32)
week_df.reset_index(inplace=True)

In [16]:
del grid_df

# 2. Feature engineering (tsfresh):
* Trends
* Autocorrelcations
* Skewness
* Fourier transforms
* Wavelet transforms
* Holiday specific features <- this in grid_df

### Creating Targets for feature selection
To pick meaningfull features we need meaningful targets

In [17]:
lift_df = pd.pivot_table(grid_df.loc[holiday_mask,['item_id', 'event_name_1', 'holiday_lift']],
               values='holiday_lift', index='item_id', columns='event_name_1', 
               aggfunc=np.mean, fill_value=0)

t = 0.8
target_df = pd.DataFrame(week_df.index)
for column in lift_df.columns:
    idx = lift_df[lift_df[column] > t].index
    target_df.loc[:, column] = week_df.item_id.isin(idx)

NameError: name 'grid_df' is not defined

In [18]:
%matplotlib inline
import matplotlib.pylab as plt

from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [19]:
# only use the first 500 ids to speed up the processing
X = extract_features(week_df, column_id="item_id", column_sort='wm_yr_wk', impute_function=impute)
y = target_df.Christmas

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
classifier_full = DecisionTreeClassifier()
classifier_full.fit(X_train, y_train)
print(classification_report(y_test, classifier_full.predict(X_test)))

Feature Extraction: 100%|██████████| 10/10 [09:46<00:00, 58.68s/it]


NameError: name 'target_df' is not defined

In [20]:
relevant_features = set()

for label in y.unique():
    y_train_binary = y_train == label
    X_train_filtered = select_features(X_train, y_train_binary)
    print("Number of relevant features for class {}: {}/{}".format(label, X_train_filtered.shape[1], X_train.shape[1]))
    relevant_features = relevant_features.union(set(X_train_filtered.columns))

len(relevant_features)

NameError: name 'y' is not defined

In [21]:
X_train_filtered = X_train[list(relevant_features)]
X_test_filtered = X_test[list(relevant_features)]

NameError: name 'X_train' is not defined

In [22]:
classifier_selected = DecisionTreeClassifier()
classifier_selected.fit(X_train_filtered, y_train)
print(classification_report(y_test, classifier_selected.predict(X_test_filtered)))

NameError: name 'X_train_filtered' is not defined

In [23]:
week_df.head()

,item_id,wm_yr_wk,sales_clean
_temporary_index_column,,,
0,FOODS_1_001,11101,0.216919
1,FOODS_1_001,11102,0.336914
2,FOODS_1_001,11103,0.590820
3,FOODS_1_001,11104,0.447754
4,FOODS_1_001,11105,0.499268


In [24]:
#y = week_df[['item_id','wm_yr_wk']].copy()
#y = (week_df.groupby(['item_id'])['sales_clean'].transform(lambda x: x.shift(-1))).astype(np.float32)
y = week_df.sales_clean

In [25]:
from tsfresh import extract_relevant_features

features_filtered_direct = extract_relevant_features(week_df, y, column_id='item_id', column_sort='wm_yr_wk')

ValueError: The following ids are in the time series container but are missing in y: {'HOBBIES_1_183', 'HOUSEHOLD_1_274', 'HOUSEHOLD_2_117', 'HOUSEHOLD_2_129', 'FOODS_3_149', 'HOUSEHOLD_1_216', 'HOUSEHOLD_1_321', 'HOUSEHOLD_2_155', 'FOODS_3_396', 'HOUSEHOLD_2_432', 'FOODS_3_775', 'HOUSEHOLD_2_410', 'FOODS_2_194', 'FOODS_3_039', 'HOBBIES_2_112', 'HOUSEHOLD_2_179', 'HOBBIES_1_070', 'FOODS_2_089', 'HOUSEHOLD_1_048', 'HOUSEHOLD_1_328', 'HOBBIES_1_284', 'FOODS_1_043', 'FOODS_2_173', 'FOODS_2_341', 'HOBBIES_1_180', 'HOBBIES_1_099', 'HOBBIES_1_110', 'HOBBIES_1_206', 'FOODS_1_121', 'FOODS_3_436', 'HOBBIES_1_229', 'FOODS_3_033', 'FOODS_3_418', 'HOUSEHOLD_1_086', 'FOODS_2_129', 'FOODS_3_567', 'FOODS_1_210', 'FOODS_3_710', 'HOBBIES_1_308', 'FOODS_3_749', 'HOUSEHOLD_1_447', 'HOUSEHOLD_2_478', 'FOODS_2_367', 'HOBBIES_1_283', 'HOUSEHOLD_1_331', 'HOUSEHOLD_1_474', 'HOUSEHOLD_1_534', 'HOUSEHOLD_1_153', 'FOODS_3_231', 'FOODS_1_031', 'FOODS_3_473', 'HOUSEHOLD_1_279', 'HOUSEHOLD_1_365', 'HOUSEHOLD_2_411'

In [26]:
# Save before we proceed as this df might be very helpful in training.
# week_df.to_pickle('m5_s1_1_01.pkl')

# Memory leaks
I am not sure whats going on, but the final week_df is just several Mb, while memory used is 10+Gb despite all the cleaning.
Apparently, there are a lot of memory leaks, yet I am not sure how to get rid of them, but dump useful info to pkl and start afresh.
Please, if you know a better solution let me know where to look.